In [1]:
import cv2
import torch
from ultralytics import YOLO

In [ ]:
model = YOLO("yolo11n.pt")

video_path = "video_path.mp4"
# cap = cv2.VideoCapture(video_path)
cap = cv2.VideoCapture(0)  # Mở camera
if not cap.isOpened():
    print("Không thể mở video hoặc camera.")
    exit()

# 3) Vòng lặp đọc từng frame
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 67 -> cell phone, conf=0.5 -> 50% confidence threshold
    results = model.predict(frame, conf=0.5, classes=[67], verbose=False)

    #    results[0].boxes chứa thông tin bboxes của frame
    for box in results[0].boxes:
        # box.xyxy: [x_min, y_min, x_max, y_max]
        # box.conf: độ tin cậy
        # box.cls : chỉ số class
        x1, y1, x2, y2 = box.xyxy[0]
        conf = float(box.conf[0])
        cls_id = int(box.cls[0])

        # Vẽ bounding box và nhãn
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(
            frame,
            f"Cell phone {conf:.2f}",
            (int(x1), int(y1) - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0, 255, 0),
            2
        )

    cv2.imshow("YOLOv8 Cell Phone Detection", frame)

    # Nhấn 'q' để thoát
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()